# Import packages

In [ ]:
import scipy as sp
import scipy.sparse as sparse

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy.sparse.linalg import svds
from sklearn.utils.extmath import randomized_svd

In [ ]:
def vec(A, stack="columns"):
    """
    vec(A) returns the vectorization of the matrix A
    by stacking the columns (or rows, respectively) of A.
    """
    if stack[0].lower() == 'c':
        return A.T.ravel()
    elif stack[0].lower() == 'r':
        return A.ravel()
    else:
        raise ValueError('Expected \'columns\' or \'rows\' for argument stack.')

In [ ]:
def unvec(vecA, shape):
    """
    _unvec(A, shape) returns the "unvectorization" of the
    matrix A by unstacking the columns of vecA to return
    the matrix A of shape shape.
    """
    return vecA.reshape(shape, order='F')

In [ ]:
def vecIndicesFromMask(mask, stack='columns'):
    """
    vecIndicesFromMask(mask, stack='columns')
    returns the vector-indices corresponding to mask == 1.
    This is operation is performed by first vectorizing the
    mask array.
    """
    return np.where(vec(mask, stack)==1)[0]

In [ ]:
def matIndicesFromMask(mask):
    """
    matIndicesFromMask(mask) returns the matrix-indices 
    corresponding to mask == 1. This operation returns a 
    tuple containing a list of row indices and a list of 
    column indices.
    """
    return np.where(mask.T==1)[::-1]

In [ ]:
def masked(A, mask):
    """
    masked(A, mask) returns the "observed entries" of the
    matrix A, as a vector, determined according to the 
    condition mask == 1 (alternatively, the entries for 
    which mask is True).
    """
    return A[matIndicesFromMask(mask)]

In [ ]:
def sparseMatComSetup(r,m,n,p):
    k = np.random.binomial(m*n, p)
    Omega = (np.random.randint(m, size=k), np.random.randint(n, size=k))
    U = np.random.randint(5, size=(m,r))
    V = np.random.randint(5, size=(n,r))
    observations = multiplyFromMatIdxList(U, V, Omega)
    M_Omega = sparse.csr_matrix((observations, Omega), 
                                shape=(m,n))
    return (U, V, Omega, observations, M_Omega)

def multiplyFromMatIdxList(U, V, Omega):
    d = Omega[0].size # nnz
    M_Omega = np.zeros(d)
    for i,j,k in zip(range(d), *Omega):
        M_Omega[i] = U[j, :] @ V[k, :]
    return M_Omega

In [ ]:
r = 3
p = .5
m = 50
n = 100

In [ ]:
U, V, Omega, obs, M_Omega = sparseMatComSetup(r, m, n, p)

In [ ]:
def matricize_right(V, Omega, m=None, sparse=True, sparse_type=None):
    """
    matricize_right(V, Omega, m=None, sparse=True, sparse_type=None) 
    turns the problem 
        M_Omega = (U @ V.T)_Omega 
    into the matrix problem
        vec(M_Omega) = W @ vec(U)
    where U is an m-by-r matrix, V is an n-by-r matrix and
        vec([[1,2,3],[4,5,6]]) = [1,4,2,5,3,6].T

    Input
              V : the right n-by-r matrix
          Omega : the mask / list of indices of observed entries
         sparse : whether to return a sparse matrix (default: true)
    sparse_type : what kind of sparse matrix to return (default: csr)

    Output
    V_op : The operator for V in matrix form so that vec(U @ V.T) is 
           equivalent to V_op @ vec(U).
    """
    if isinstance(Omega, tuple):
        Omega_i = Omega[0]
        Omega_j = Omega[1]
        if m is None:
            raise ValueError('input number of columns for left' +
                             ' factor is required when Omega is a ' +
                             'list of indices')
    elif isinstance(Omega, np.ndarray):
        m = Omega.shape[0]
        Omega_i, Omega_j = matIndicesFromMask(Omega)
    else:
        raise ValueError('type of Omega not recognized; ' + 
                         'expected tuple of indices or mask array.')
    r = V.shape[1]
    sizeU = m*r
    if sparse:
        sp_mat = _get_sparse_type(sparse_type)
        row_idx = np.repeat(range(Omega_i.size), r)
        col_idx = [np.arange(Omega_i[n], sizeU, m, dtype=int) 
                   for n in range(Omega_i.size)]
        col_idx = np.concatenate(col_idx)
        vals = np.concatenate([V[j,:] for j in Omega_j])
        V_op = sp_mat((vals, (row_idx, col_idx)), shape=(Omega_i.size, sizeU))
    else:
        V_op = np.zeros((Omega_i.size, sizeU))
        for n in range(Omega_i.size):
            i = Omega_i[n]
            j = Omega_j[n]
            V_op[n, i::m] = V[j,:]
    return V_op

In [ ]:
def matricize_left(U, Omega, n=None, sparse=True, sparse_type=None):
    """
    matricize_left(U, Omega, n=None, sparse=True, sparse_type=None) 
    turns the problem
        M_Omega = (U @ V.T)_Omega
    into the matrix problem
        vec(M_Omega) = W @ vec(V)
    where U is an m-by-r matrix, V is an n-by-r matrix and
        vec([[1,2,3],[4,5,6]]) = [1,4,2,5,3,6].T

    Input
              U : the left m-by-r matrix
          Omega : the mask / list of indices of observed entries
         sparse : whether to return a sparse matrix (default: true)
    sparse_type : what kind of sparse matrix to return (default: csr)

    Output
    U_op : The operator for U in matrix form so that vec(U @ V.T) is 
           equivalent to U_op @ vec(V).
    """
    if isinstance(Omega, tuple):
        Omega_i = Omega[0]
        Omega_j = Omega[1]
        if n is None:
            raise ValueError('input number of columns for right' +
                             ' factor is required when Omega is a ' +
                             'list of indices')
    elif isinstance(Omega, np.ndarray):
        n = Omega.shape[1]
        Omega_i, Omega_j = matIndicesFromMask(Omega)
    else:
        raise ValueError('type of Omega not recognized; ' + 
                         'expected tuple of indices or mask array.')

    r = U.shape[1]
    sizeV = n*r

    if sparse:
        sp_mat = _get_sparse_type(sparse_type)
        row_idx = np.repeat(range(Omega_j.size), r)
        col_idx = [np.arange(Omega_j[idx], sizeV, n, dtype=int) 
                   for idx in range(Omega_j.size)]
        col_idx = np.concatenate(col_idx)
        vals = np.concatenate([U[i,:] for i in Omega_i])
        U_op = sp_mat((vals, (row_idx, col_idx)), shape=(Omega_j.size, sizeV))
    else:
        U_op = np.zeros((Omega_j.size, sizeV))
        for idx in range(Omega_j.size):
            i = Omega_i[idx]
            j = Omega_j[idx]
            U_op[idx, j::n] = U[i,:]
    return U_op